In [68]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from nltk import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [69]:
#Descargar el conjunto de datos del tokenizador en español
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Darío\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [70]:
df = pd.read_csv('Base.csv')

In [71]:
df.head()

,Unnamed: 0,indi,cuerpo,titular
0,0,0,dos semanas después de su puesta de largo y pr...,el submarino s-80 ya flota
1,1,1,"este viernes, el presidente del gobierno, pedr...",calviño y calvo alaban (sin darse cuenta) la g...
2,2,2,"el ministro del interior, fernando grande-marl...",el geo de la policía tendrá una nueva sede en ...
3,3,3,son días muy duros para la familia de olivia y...,"la madre de las niñas ""sobran las palabras par..."
4,4,4,sólo quedan 10 presos de eta por recibir los b...,sólo quedan 10 presos de eta por recibir el be...


In [72]:
textos = df['cuerpo']
textos.describe()

count                                                 58424
unique                                                43468
top       entre clase y clase en la facultad de ciencias...
freq                                                   1297
Name: cuerpo, dtype: object

In [73]:
probs = {} # key: (w(t-1), w(t+1)), value: {w(t): count(w(t))}

In [74]:
for doc in tqdm(textos):
    lineas = doc.split('.')
    for linea in lineas:
        tokens = word_tokenize(linea, language='spanish')
        if len(tokens) >=2:
            for i in range(len(tokens) - 2):
                t_0 = tokens[i]
                t_1 = tokens[i+1]
                t_2 = tokens[i+2]
                key = (t_0, t_2)
                if key not in probs:
                    probs[key] = {}
                if t_1 not in probs[key]:
                    probs[key][t_1] = 1
                else:
                    probs[key][t_1] += 1

100%|██████████| 58424/58424 [03:21<00:00, 289.28it/s]


In [75]:
# normalicemos probabilidades
for key, d in probs.items():
    total = sum(d.values())
    for k, v in d.items():
        d[k] = v / total

In [76]:
detokenizar = TreebankWordDetokenizer()

In [77]:
def spin_document(doc):
    lineas = doc.split('.')
    output = []
    for linea in lineas:
        if linea:
            new_line = spin_line(linea)
        else:
            newline = linea
        output.append(new_line)
    return '\n'.join(output)

In [78]:
def sample_word(d):
    p0 = np.random.random()
    cumulative = 0
    for t, p in d.items():
        cumulative += p
        if p0 < cumulative:
            return t
        
# extrae una palabra al azar si le damos una clave para buscar en el diccionario probs

In [79]:
def spin_line(linea):
    tokens = word_tokenize(linea, language='spanish')
    i = 0
    salida = [tokens[0]]
    if len(tokens) >= 2:
        while i < (len(tokens) - 2):
            t_0 = tokens[i]
            t_1 = tokens[i+1]
            t_2 = tokens[i+2]
            key = (t_0, t_2)
            p_dist = probs[key]
            if len(p_dist) > 1 and np.random.random() < 0.3:
                middle = sample_word(p_dist)
                salida.append(t_1)
                salida.append('<' + middle + '>')
                salida.append(t_2)
                i += 2
            else:
                salida.append(t_1)
                i += 1
        if i == len(tokens) - 2:
            salida.append(tokens[-1])
        return detokenizar.detokenize(salida)

In [80]:
i = np.random.choice(textos.shape[0])   
doc = textos.iloc[i] # selecciono un segmento random del documento para hacerle el spin
new_doc = spin_document(doc)
new_doc

'el ayuntamiento <22> de madrid <madrid> establecerá una prioridad sobre las obras pendientes y evaluará cómo "alargar <funcionar> en el tiempo <premio>" la ejecución de "todos <evidencia>" los <los> proyectos, <tiene> entre los que se encuentran <elimina> la reforma de la <una> gran vía <responsabilidad> y de la <la> plaza de <de> españa, "<tanto> en el caso de que <que> sea necesario" <blindar> por la intervención de las <las> cuentas municipales decretada por el <el> ministerio de hacienda\nla portavoz <victoria> del ejecutivo municipal <central>, rita maestre, en la rueda de prensa posterior a la junta <junta> de gobierno <mantener>, ha <ha> asegurado que <que> "no hay ninguna obra <operación> que se queda sin hacer <precedentes>"\nmaestre ha contestado así después <como> de ser <ser> preguntada si tendrá <en> algún coste para obras como la <la> prevista en gran vía, por ejemplo, <,> el requerimiento del ministerio <presupuesto> de hacienda de tutelar semanalmente las operaciones f